 # Notebook for Downloading RDA Data.

In [1]:
import sys, os
import requests

In [2]:
filelist = [ "https://request.rda.ucar.edu/dsrqst/BASU802497/TarFiles/gfs.0p25.2025060212.f084-25.2025071500.f018.grib2.nc.tar",
 ]

Change the value of `save_dir` if you prefer your files saved somewhere other than the current directory.

In [3]:
save_dir = '/home/subhojit/sanghita_14-06-2025/GFS/U850'

 ## Now to download the files

In [4]:
for file in filelist:
    print('Downloading:', file)

    try:
        # Use URL directly for the request
        req = requests.get(file, allow_redirects=True)

        # Save using just the filename part
        filename = os.path.join(save_dir, os.path.basename(file))
        with open(filename, 'wb') as f:
            f.write(req.content)

        print(f"Saved to: {filename}")
    except Exception as e:
        print(f"Failed to download {file}: {e}")

Downloading: https://request.rda.ucar.edu/dsrqst/BASU802497/TarFiles/gfs.0p25.2025060212.f084-25.2025071500.f018.grib2.nc.tar
Saved to: /home/subhojit/sanghita_14-06-2025/GFS/U850/gfs.0p25.2025060212.f084-25.2025071500.f018.grib2.nc.tar


### Once you have downloaded the data, the next part can help you plot it.

In order to plot this data, you may need to install some libraries. The easiest way to do this is to use conda or pip, however any method of getting the following libraries will work.

In [ ]:
import xarray # used for reading the data.
import matplotlib.pyplot as plt # used to plot the data.
import ipywidgets as widgets # For ease in selecting variables.
import cartopy.crs as ccrs # Used to georeference data.

In [ ]:
filelist_arr = [save_dir + os.path.basename(file) for file in filelist]
selected_file = widgets.Dropdown(options=filelist_arr, description='data file')
display(selected_file)

In [ ]:
# Now to load in the data to xarray
ds = xarray.open_dataset(selected_file.value)

In [ ]:
# Helper methods# Define function to get standard dimensions
def get_primary(dataset):
    primary_variables = {}
    coords = dataset.coords.keys()
    highest_dims = 0
    for cur_key,cur_var in dataset.variables.items():
        if cur_key not in coords:
            primary_variables[cur_key] = cur_var
    return primary_variables 

In [ ]:
var = widgets.Dropdown(
    options=get_primary(ds).keys(),
    description='Variable')
display(var)

In [ ]:
proj = ccrs.Mercator()
plt.gcf().set_size_inches(20,10)
ax = plt.axes(projection=proj)
data_slice = ds[var.value].isel(time=0)
data_slice.plot.contourf(ax=ax, transform=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()